In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
market = Market()
competition_db = ADatabase("competition")

In [4]:
market.connect()
prices = market.retrieve("prices")
sp5 = market.retrieve("sp500")
market.disconnect()

In [5]:
prices = p.column_date_processing(prices)

In [6]:
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]

In [7]:
prices["close"] = [float(x) for x in prices["close"]]

In [8]:
data = prices.groupby(["year","week","ticker"]).mean()
data = prices.pivot_table(index=["year","week"],columns="ticker",values="close").reset_index()

In [9]:
data = data[data["year"]>2010]
data.head()

ticker,year,week,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,...,WST,WTW,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBRA
887,2011,1,41.574,11.0020,62.442,332.9420,NaN,34.502,9.9140,48.1800,...,41.1980,NaN,20.012,113.7240,23.590,74.984,34.8780,NaN,48.8860,37.7840
888,2011,2,42.904,11.0000,62.656,344.5350,NaN,35.094,10.1740,47.5240,...,40.7000,NaN,21.290,117.6220,23.500,76.390,35.8505,NaN,48.9500,38.4460
889,2011,3,42.795,10.0575,63.260,334.7225,NaN,35.885,9.7325,47.6025,...,39.7075,NaN,21.780,117.3075,23.840,78.420,36.1200,NaN,48.8625,38.3825
890,2011,4,42.292,10.4480,63.990,340.4020,NaN,36.290,9.9260,46.9500,...,40.1380,NaN,22.858,116.4480,23.932,79.160,36.0200,NaN,47.6120,38.8020
891,2011,5,41.818,9.6360,63.072,343.7220,NaN,35.924,11.0660,45.6480,...,40.4960,NaN,23.704,118.3680,23.700,82.944,36.1025,NaN,48.0880,39.8260


In [10]:
data.fillna(-99999,inplace=True)
for col in data.columns:
    if -99999 == data[col].min():
        data.drop(col,inplace=True,axis=1)

In [12]:
competition_db.connect()
competition_db.store("data",data)
competition_db.disconnect()